In [1]:
# IMAP allows you to access your email wherever you are, from any device. 
# When you read an email message using IMAP, you aren't actually downloading or storing it on your computer; 
# instead, you're reading it from the email service.

# The OS module in Python provides functions for interacting with the operating system. 

import imaplib, email, os

In [6]:
# setup your account details and connect with imap service
user = 'test-071@outlook.com'
password = 'Testingaccount'
imap_url = 'imap-mail.outlook.com'

In [7]:
# Specify the path where you want to save your attachments

attachment_dir = r'C:\Users\karis\Documents\PATH\email scraping for Outlook and Gmail'

In [8]:
def auth(user,password,imap_url):
    con = imaplib.IMAP4_SSL(imap_url)
    con.login(user,password)
    return con

In [9]:
con = auth(user,password,imap_url)
con.select('Inbox')

('OK', [b'7'])

In [10]:
def get_attachments(msg):
    for part in msg.walk():
        if part.get_content_maintype() == 'multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue 
        fileName = part.get_filename()
        
        if  bool(fileName):
            filePath = os.path.join(attachment_dir, fileName)
            with open(filePath, 'wb') as f:
                f.write(part.get_payload(decode=True))

In [11]:
con.list()

('OK',
 [b'(\\HasNoChildren) "/" Archive',
  b'(\\HasNoChildren \\Trash) "/" Deleted',
  b'(\\HasNoChildren \\Drafts) "/" Drafts',
  b'(\\Marked \\HasNoChildren) "/" Inbox',
  b'(\\HasNoChildren \\Junk) "/" Junk',
  b'(\\HasNoChildren) "/" Notes',
  b'(\\HasNoChildren) "/" Outbox',
  b'(\\HasNoChildren \\Sent) "/" Sent'])

In [12]:
con.select('Inbox')

('OK', [b'7'])

In [13]:
result, data = con.fetch(b'2','(RFC822)')

In [14]:
data
#you can ^F to find 'attach' and see if the data from the email has an attachment

[(b'2 (RFC822 {3341767}',
  b'Received: from DM6NAM04HT142.eop-NAM04.prod.protection.outlook.com\r\n (2603:10b6:408:f6::7) by BN8PR10MB3457.namprd10.prod.outlook.com with HTTPS\r\n via BN9PR03CA0332.NAMPRD03.PROD.OUTLOOK.COM; Thu, 5 Aug 2021 14:15:59 +0000\r\nARC-Seal: i=2; a=rsa-sha256; s=arcselector9901; d=microsoft.com; cv=pass;\r\n b=eANM9n7ptPxImrbNvvACWJFDfzTOMCv506zyLyfVYS14j3HNKgRXAbYS931pzIh5Zfp0QBD75JLphkPx5w3L/PLn4FgtB5+1TlUocmdr2ePe7QIFfxc40jo0Xx1xUyyLf6qoLXATDrmHiiXiJ0Q6qtpx3YmbiPgvceB6uChMvL8lAUatud412UOsruJnx7w954DzcXCDnGEK4qY2xreKIjkqMBG/8cEIE3FYnNqPyu54SCmn9ft9cYxwb3dSTEm/XkYQVFBnY+IuvHMIdYzMJ1Bbpe2oqNvkOWZuslZ3dr0ri054lpPAQK5ob0CCit2cKy2U6Lzf56TAZSg22I+amg==\r\nARC-Message-Signature: i=2; a=rsa-sha256; c=relaxed/relaxed; d=microsoft.com;\r\n s=arcselector9901;\r\n h=From:Date:Subject:Message-ID:Content-Type:MIME-Version:X-MS-Exchange-SenderADCheck;\r\n bh=2fsS6X8T2560FIUew6sGXAEExCiX2iV/GylIrNWj8Jw=;\r\n b=gsj4I39JN60AACI5C/nEQOuOD6wENd+QYc807YE76WfgzKhBt4wC8lHr6AmFpx

In [15]:
raw = email.message_from_bytes(data[0][1])
get_attachments(raw)